In [1]:
import pandas as pd
import pickle
import numpy as np

# Load the dataset
dataset_PV_Pred = pd.read_csv('Dataset_PV_Pred.csv')

Hour_Parameters = pd.read_csv('Parametros_Horarios_SVR+SA_PV_NorteShopping(Hist(12)+Temp(Hist+Prev)).csv')
Hour_Parameters.columns.values[0] = 'Hour'


# Load the SVR model
with open('SVR_Model.pkl', 'rb') as file:
    svr_model = pickle.load(file)


c:\Users\tomas.m.santos\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator SVR from version 1.4.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
Predicted_Values = []

for h in range(0, 24):
    # Select rows where 'Hora' is equal to h
    Inputs_hora = dataset_PV_Pred.loc[dataset_PV_Pred['Hora'] == h].iloc[:, 2:]
    Outputs_hora = dataset_PV_Pred.loc[dataset_PV_Pred['Hora'] == h].iloc[:, 1]
    # Set SVR model parameters for the current Hora
    svr_model.set_params(
        C=Hour_Parameters.loc[Hour_Parameters['Hour'] == h, 'C'].values[0],
        gamma=Hour_Parameters.loc[Hour_Parameters['Hour'] == h, 'gamma'].values[0],
        epsilon=Hour_Parameters.loc[Hour_Parameters['Hour'] == h, 'epsilon'].values[0],
        kernel='rbf'
    )

    # Prepare the input data for prediction
    X = Inputs_hora[['Lag51', 'Lag50', 'Lag49', 'Lag48', 'Lag27', 'Lag26', 'Lag25', 'Lag24', 'Lag3', 'Lag2', 'Lag1', 'Temperatura_24H', 'Temperatura_Prev']]
    X = X.values.reshape(-1, 13)  # Reshape to 2D array for prediction
    Y = Outputs_hora
    # Fit the SVR model to the data
    svr_model.fit(X, Y) 
    # Predict using the SVR model
    Y_pred = svr_model.predict(X)
    # Append the predicted values to the list
    Predicted_Values.append(Y_pred)


Previsoes = np.array(Predicted_Values).T.reshape(-1, 1)
print(Previsoes.shape)

(17136, 1)


In [3]:
Previsoes = pd.DataFrame(Previsoes, columns=['Previsao'])
Previsoes

,Previsao
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
17131,-1.925587
17132,0.238973
17133,0.000000
17134,0.000000
